In [ ]:
import sys
import re
import gzip
from collections import Counter

## annot.py
- oheisella scriptillä poimitaan conllu-tiedostosta vain Suomi24-kommenttien annotaatiot,
- tuloksena seuraavissa vaiheissa hyödynnettävä txt-tiedosto
- notebookia varten with open, terminaalissa helpompi

In [ ]:
def read_text(inp):
    id = None
    anno_rivit = []
    for line in inp:
        line = line.strip()
        if line.startswith("ID:"):
            if id is not None:
                yield (id, anno_rivit)
            id = line
            anno_rivit = []
        elif id is None:
            id = line
        else:
            if line.startswith("TEKSTI:") or line.startswith("ANNOT:") or line.startswith("LABEL:"):
                continue
            else:
                if line:
                    annot = line
                    anno_rivit.append(annot)

    yield (id, anno_rivit)

input_tiedosto = ""  # tähän paikkakuntakorpus
output_tiedostonimi = "annot.txt"
with open(input_tiedosto, "r") as input_f, open(output_tiedostonimi, "w") as output_f:
    for id, anno_rivit in read_text(input_f):
        for annot in anno_rivit:
            output_f.write(annot + "\n")


- ohjeet seuraavien skriptien tulkitsemiseen tarvittaessa erillisessä tiedostossa (OHJE)
- käytännössä tiedostosta etsitään tiettyjen lekseemien kaikki vaihtelua sisältävät muodot (edellytyksenä etsittavat_sanat-muuttujassa määritelty lemma), ja lisätään ne listoihin
- ohjeessa ja osassa vaihteluskriptejä on käytetty system input tiedostojen käsittelyyn, tarvittaessa pitää vaihtaa


##  /ts/ vaihtelu

In [ ]:
etsittavat_sanat = ["kahtoa", "katsoa", "kattoa", "itte", "itse", "ihte", "ruotti", "ruotsi", "ruohti", ]

tt_vaihtelu = []
ts_vaihtelu = []
t_vaihtelu = []
ht_vaihtelu = []
muu = []

laskuri = Counter()

def annojen_erittely(line):
    pattern = r"'.*?'"
    annot = re.findall(pattern, line)
    annot = [i.strip("'") for i in annot]
    return annot

with open("annot.txt", "r") as f:
    for line in f:
        annorivi = annojen_erittely(line)
        if len(annorivi) >= 2:
            etsittava_sana = annorivi[2].lower()
            if etsittava_sana in etsittavat_sanat:
                if "ts" in annorivi[1].lower():
                    ts_vaihtelu.append(annorivi)
                elif "tt" in annorivi[1].lower():
                    tt_vaihtelu.append(annorivi)
                elif "ht" in annorivi[1].lower():
                    ht_vaihtelu.append(annorivi)
                elif "t" in annorivi[1].lower() and "h" not in annorivi[1].lower() and "ts" not in annorivi[1].lower() and "tt" not in annorivi[1].lower():
                    t_vaihtelu.append(annorivi)
                else:
                    muu.append(annorivi)

# lasketaan kaikki esiintymät ja osuudet
total_matches = len(tt_vaihtelu) + len(ts_vaihtelu) + len(t_vaihtelu) + len(ht_vaihtelu) + len(muu)

if total_matches > 0:
    percentages = {
        "tt_vaihtelu": len(tt_vaihtelu) / total_matches * 100,
        "ts_vaihtelu": len(ts_vaihtelu) / total_matches * 100,
        "t_vaihtelu": len(t_vaihtelu) / total_matches * 100,
        "ht_vaihtelu": len(ht_vaihtelu) / total_matches * 100,
        "muu": len(muu) / total_matches * 100,
    }

    for key, value in percentages.items():
        print(f"{key} osuus esiintymistä: {value:.2f}%\n")

print("Kaikki tt esiintymät:", len(tt_vaihtelu))
for item in tt_vaihtelu:
    print(item)
print("Kaikki ts esiintymät:", len(ts_vaihtelu))
for item in ts_vaihtelu:
    print(item)
print("Kaikki t+kato esiintymät:", len(t_vaihtelu))
for item in t_vaihtelu:
    print(item)
print("Kaikki ht esiintymät:", len(ht_vaihtelu))
for item in ht_vaihtelu:
    print(item)
print("Muut esiintymät:", len(muu))
for item in muu:
    print(item)


## metsä-sanan vaihtelu
- voisi yhdistää myös ts-vaihtelun tarkasteluun
- Huom. system input --> korvaa tarvittaessa open file

In [ ]:
etsittavat_sanat = ["metsä", "mettä", "mehtä", "mehttä", "messä"]

tt_vaihtelu = []
ts_vaihtelu = []
t_vaihtelu = []
ht_vaihtelu = []
ss_vaihtelu = []
muu = []

laskuri = Counter()

def annojen_erittely(line):
    pattern = r"'.*?'"
    annot = re.findall(pattern, line)
    annot = [i.strip("'") for i in annot]
    return annot

for line in sys.stdin:
    annorivi = annojen_erittely(line)
    if len(annorivi) >= 2:
        etsittava_sana = annorivi[2].lower()
        if etsittava_sana in etsittavat_sanat:
            if "ts" in annorivi[1].lower():
                ts_vaihtelu.append(annorivi)
            elif "tt" in annorivi[1].lower():
                tt_vaihtelu.append(annorivi)
            elif "ht" in annorivi[1].lower():
                ht_vaihtelu.append(annorivi)
            elif "ss" in annorivi[1].lower() and "ts" not in annorivi[1].lower():
                ss_vaihtelu.append(annorivi)
            elif "t" in annorivi[1].lower() and "h" not in annorivi[1].lower() and "ts" not in annorivi[1].lower() and "tt" not in annorivi[1].lower():
                t_vaihtelu.append(annorivi)
            else:
                muu.append(annorivi)

# lasketaan kaikki esiintymät ja osuudet
total_matches = len(tt_vaihtelu) + len(ts_vaihtelu) + len(t_vaihtelu) + len(ht_vaihtelu) + len(muu)

if total_matches > 0:
    percentages = {
        "tt_vaihtelu": len(tt_vaihtelu) / total_matches * 100,
        "ts_vaihtelu": len(ts_vaihtelu) / total_matches * 100,
        "ss_vaihtelu": len(ss_vaihtelu) / total_matches * 100,
        "t_vaihtelu": len(t_vaihtelu) / total_matches * 100,
        "ht_vaihtelu": len(ht_vaihtelu) / total_matches * 100,
        "muu": len(muu) / total_matches * 100,
    }

    for key, value in percentages.items():
        print(f"{key} osuus esiintymistä: {value:.2f}%\n")

# printataan löydökset
print("Kaikki tt esiintymät:", len(tt_vaihtelu))
print("Kaikki ts esiintymät:", len(ts_vaihtelu))
print("Kaikki ss esiintymät:", len(ss_vaihtelu))
for item in ss_vaihtelu:
    print(item)
print("Kaikki t+kato esiintymät:", len(t_vaihtelu))
print("Kaikki ht esiintymät:", len(ht_vaihtelu))
print("Muut esiintymät:", len(muu))
for item in muu:
    print(item)


## /d/ vaihtelu
- Huom. system input --> korvaa tarvittaessa open file

In [ ]:
etsittavat_sanat = ["syödä", "syydä", "syörä", "syyvä", "syvvä", "kären", "käsi", "käjen", "rauva", "rauta", "raura", "tehrä", "tehä",]

yleiskielinen = []
lantinen_r = []
itainen_v = []
muu = []

laskuri = Counter()

def annojen_erittely(line):
    pattern = r"'.*?'"
    annot = re.findall(pattern, line)
    annot = [i.strip("'") for i in annot]
    return annot

for line in sys.stdin:
    annorivi = annojen_erittely(line)
    if len(annorivi) >= 2:
        etsittava_sana = annorivi[2].lower()
        if etsittava_sana in etsittavat_sanat:
            if "d" in annorivi[1].lower():          # yleiskielinen d
                yleiskielinen.append(annorivi)
            elif len(etsittava_sana) >= 4 and "r" in etsittava_sana[2].lower() or "r" in etsittava_sana[3].lower() : # läntinen r
                lantinen_r.append(annorivi)
            elif "v" in annorivi[1].lower() and "yö" not in etsittava_sana:   # itäinen v
                itainen_v.append(annorivi)
            else:
                muu.append(annorivi)

# lasketaan kaikki esiintymät ja osuudet
total_matches = len(lantinen_r) + len(yleiskielinen) + len(itainen_v) + len(muu)

if total_matches > 0:
    percentages = {
    "Läntinen r vaihtelu": len(lantinen_r) / total_matches * 100,
    "Yleiskielinen d": len(yleiskielinen) / total_matches * 100,
    "Itäinen v vaihtelu": len(itainen_v) / total_matches * 100,
    "Muut esiintymät": len(muu) / total_matches * 100,
}

    for key, value in percentages.items():
        print(f"{key} osuus esiintymistä: {value:.2f}%\n")

# esiintymät näkyviin, poista # tarvittaessa
#print("Kaikki r vaihtelut:", len(lantinen_r))
#for item in lantinen_r:
#    print(item)
# print("Kaikki yleiskieliset variantit:", len(yleiskielinen))
# for item in yleiskielinen:
#    print(item)
#print("Kaikki itäiset v:"), len(itainen_v)
#for item in itainen_v:
#    print(item)
#print("Kaikki muut esiintymät:", len(muu))
#for item in muu:
#   print(item)

## Inessiivi
- Nimitykset voisi katsoa uudelleen. Esimerkiksi "supistuminen" ei sopiva tässä kohtaa
- Huom. system input --> korvaa tarvittaessa open file

In [ ]:
etsittavat_sanat = ["koulu", "koulus", "koulusa", "paikka", "paikas", "paikasa", "paikaisin", "hukassa", "hukas", "hukasa", "hukka", "töissä", "töis", "töisä",]

yleiskielinen = []
lantinen_kato = []
lantinen_s = []
muu = []   # tänne kerätään muut muodot

laskuri = Counter()

def annojen_erittely(line):
    pattern = r"'.*?'"
    annot = re.findall(pattern, line)
    annot = [i.strip("'") for i in annot]
    return annot

for line in sys.stdin:
    annorivi = annojen_erittely(line)
    if len(annorivi) >= 2:
        etsittava_sana = annorivi[2].lower()
        if etsittava_sana in etsittavat_sanat:
          if "kk" not in annorivi[1]: # tällä varmistetaan inessiivit koska kk:k vaihtelu
              if "ssa" in annorivi[1].lower():
                  yleiskielinen.append(annorivi)
              elif "sa" in (annorivi[1]).lower() and "ss" not in annorivi[1].lower() and "Case=Ine" in annorivi[5]:
                  lantinen_s.append(annorivi)
              elif "s" in (annorivi[1])[-1].lower():
                  lantinen_kato.append(annorivi)
              else:
                  muu.append(annorivi)

# prosentteja varten
total_matches = len(yleiskielinen) + len(lantinen_kato) + len(lantinen_s)

if total_matches > 0:
    percentages = {
        "Yleiskielinen pääte": len(yleiskielinen) / total_matches * 100,
        "Muut läntiset s-päätteet (kato)": len(lantinen_kato) / total_matches * 100,
        "Päätteenä kaksoiskonsonantin supistuminen eli paikasa": len(lantinen_s) / total_matches * 100,
    }

    for key, value in percentages.items():
        print(f"{key} osuus esiintymistä: {value:.2f}%\n")

# esiintymät ulos, taman voi poistaa jos tarve vaan frekvensseille
print("Kaikki yleiskieliset esiintymät:", len(yleiskielinen))
print("Ei eritellä yleiskielisiä esiintymiä")
print("Muut läntiset s-päätteet (kato):", len(lantinen_kato))
for item in lantinen_kato:
    print(item)
print("Kaikki kaksoiskonsonantin supistumiset eli paikasa:", len(lantinen_s))
for item in lantinen_s:
    print(item)
print("Muut esiintymät:", len(muu))
print("Ei eritellä muita")


## Kieltohakuiset määrän partikkelimaiset adverbit
- Mielenkiinnosta näiden esiintymät korpuksessa
- Lemmoja voisi lisätä, ja toisaalta näitäkin voisi jakaa eri luokkiin eikä tarkastella yhdessä "adverbiesiintymänä"
- Huom. system input --> korvaa tarvittaessa open file

In [ ]:
import sys
import re
from collections import Counter

etsittavat_sanat = ["lainkaan", "ollenkaan", "ollaikaa", "ollenka", "ollenkaa", "ensinkään", "ensinkää", "ensinkä", "ensinkkä", "laisinkaan", "lainenka", "laisinka",]

adverbiesiintyma = []
muu = []   # tänne kerätään muut muodot

laskuri = Counter()
kaikkilemmat = 0

def annojen_erittely(line):
    pattern = r"'.*?'"
    annot = re.findall(pattern, line)
    annot = [i.strip("'") for i in annot]
    return annot

for line in sys.stdin:
    annorivi = annojen_erittely(line)
    if len(annorivi) >= 2:
        kaikkilemmat += 1
        etsittava_sana = annorivi[2].lower()
        if etsittava_sana in etsittavat_sanat:
          adverbiesiintyma.append(annorivi)

if kaikkilemmat > 0:
  osuus = round((len(adverbiesiintyma) / kaikkilemmat) * 100, 4)
else:
  osuus = 0

# esiintymät ulos, taman voi poistaa jos tarve vaan frekvensseille
print("Adverbit, joita etsittiin:", len(adverbiesiintyma))
print("Yhteensä lemmoja:", kaikkilemmat)
print("Eli osuus kaikista:", osuus, "%")
for item in adverbiesiintyma:
    print(item)

## Persoonapronominit
- Huom. lemmana minä/sinä on huomioitu, sillä osa murteellisista wordformeista on annotoitu yleiskielisiksi. Luokittelussa näitä ei kuitenkaan ole vielä kerätty omaan listaan
- Huom. system input --> korvaa tarvittaessa open file

In [ ]:
etsittavat_sanat = ["mä", "mää", "sä", "sää", "mie", "miä", "sie", "siä", "minä", "sinä"]

mä_sä = []
mie_sie = []
muu = []

laskuri = Counter()

def annojen_erittely(line):
    pattern = r"'.*?'"
    annot = re.findall(pattern, line)
    annot = [i.strip("'") for i in annot]
    return annot

for line in sys.stdin:
    annorivi = annojen_erittely(line)
    if len(annorivi) >= 2:
        etsittava_sana = annorivi[2].lower()
        if (etsittava_sana in etsittavat_sanat and
            "PRON" in annorivi[3] and
            "l" not in annorivi[1] and
            len(annorivi[1]) > 1 and
            any(word in annorivi[1] for word in ["mä", "sä", "mie", "sie"])
           ):
            if "mä" in etsittava_sana or "sä" in etsittava_sana:
                mä_sä.append(annorivi)
            elif "mie" in etsittava_sana or "sie" in etsittava_sana:
                mie_sie.append(annorivi)

# lasketaan kaikki esiintymät ja osuudet
total_matches = len(mä_sä) + len(mie_sie) + len(muu)

if total_matches > 0:
    percentages = {
        "Läntinen mä tai sä": len(mä_sä) / total_matches * 100,
        "Itäinen mie tai sie": len(mie_sie) / total_matches * 100,
        "Muut esiintymät": len(muu) / total_matches * 100,
    }

    for key, value in percentages.items():
        print(f"{key} osuus esiintymistä: {value:.2f}%\n")

# esiintymät ulos, taman voi poistaa jos tarve vaan frekvensseille
print("Kaikki läntiset:", len(mä_sä))
for item in mä_sä:
    print(item)
print("Kaikki itäiset:", len(mie_sie))
for item in mie_sie:
    print(item)
print("Kaikki muut esiintymät:", len(muu))
for item in muu:
    print(item)


Kaikki läntiset: 0
Kaikki itäiset: 0
Kaikki muut esiintymät: 0


## Diftongien avartuminen

In [ ]:
etsittavat_sanat = ["tyä", "työ", "tyähätä", "hiano", "hieno", "kiero", "kiaro", "lyö", "lyä", "syö", "syä", "syädä", "syödä",]

avartuminen = []
yleiskielinen = []
muu = []

laskuri = Counter()

def annojen_erittely(line):
    pattern = r"'.*?'"
    annot = re.findall(pattern, line)
    annot = [i.strip("'") for i in annot]
    return annot

for line in sys.stdin:
    annorivi = annojen_erittely(line)
    if len(annorivi) >= 2:
        etsittava_sana = annorivi[2].lower()
        if etsittava_sana in etsittavat_sanat:
            if "uo" or "yö" or "ie" in annorivi[1]:
                yleiskielinen.append(annorivi)
            elif "ua" or "yä" or "ia" in annorivi[1]:
                avartuminen.append(annorivi)
            else:
                muu.append(annorivi)

# lasketaan kaikki esiintymät ja osuudet
total_matches = len(avartuminen) + len(yleiskielinen) + len(muu)

if total_matches > 0:
    percentages = {
        "Yleiskielinen": len(yleiskielinen) / total_matches * 100,
        "Diftongien avartuminen": len(avartuminen) / total_matches * 100,
        "Muut esiintymät": len(muu) / total_matches * 100,
    }

    for key, value in percentages.items():
        print(f"{key} osuus esiintymistä: {value:.2f}%\n")
    print()

# esiintymät ulos, taman voi poistaa jos tarve vaan frekvensseille
print("Kaikki yleiskieliset variantit:", len(yleiskielinen))
for item in yleiskielinen:
    print(item)
print("Kaikki diftongien avartumiset:", len(avartuminen))
for item in avartuminen:
    print(item)
print("Kaikki muut esiintymät:", len(muu))
for item in muu:
    print(item)

## Länsimurteiden lyhentyneet vokaalit

In [ ]:
etsittavat_sanat = ["jiähtyä", "jähtyä", "jäähtyä", "hahmo", "haahmo", "huahmo", "pyhkii", "pyyhkii", "pyihkii", "ruhka", "ruuhka"]

lyhentynyt_vokaali = []
pitka_vokaali = []
muu_itainen = []
muu = []

laskuri = Counter()

def annojen_erittely(line):
    pattern = r"'.*?'"
    annot = re.findall(pattern, line)
    annot = [i.strip("'") for i in annot]
    return annot

for line in sys.stdin:
    annorivi = annojen_erittely(line)
    if len(annorivi) >= 2:
        etsittava_sana = annorivi[2].lower()
        if etsittava_sana in etsittavat_sanat:
            if len(etsittava_sana) > 1 and etsittava_sana[1] not in "aeiouyäö": # jos sanan toinen kirjain ei vokaali -> muu
                muu.append(annorivi)
            elif len(etsittava_sana) >= 4 and etsittava_sana[2] in "aeiouyäö" and etsittava_sana[3] not in "aeiouyäö": # jos toinen on vokaali, mutta kolmas ei -> lyhentynyt
                lyhentynyt_vokaali.append(annorivi)
            elif len(etsittava_sana) >= 4 and etsittava_sana[2] in "aeiouyäö" and etsittava_sana[3] in "aeiouyäö": # jos kaksi vokaalia ->
                if etsittava_sana[2] == etsittava_sana[3]: # jos pitkä vokaali
                    pitka_vokaali.append(annorivi)
                else:  # jos diftongi yms.
                    muu_itainen.append(annorivi)

# lasketaan kaikki esiintymät ja osuudet
total_matches = len(pitka_vokaali) + len(muu_itainen) + len(lyhentynyt_vokaali) + len(muu)

if total_matches > 0:
    percentages = {
        "Lyhentynyt vokaali": len(lyhentynyt_vokaali) / total_matches * 100,
        "Pitkä vokaali": len(pitka_vokaali) / total_matches * 100,
        "Muu itainen eli diftongi": len(muu_itainen) / total_matches * 100,
        "Muut esiintymät": len(muu) / total_matches * 100,
    }

    for key, value in percentages.items():
        print(f"{key} osuus esiintymistä: {value:.2f}%\n")

# esiintymät ulos, taman voi poistaa jos tarve vaan frekvensseille
print("Kaikki lyhentyneet vokaalit:", len(lyhentynyt_vokaali))
for item in lyhentynyt_vokaali:
    print(item)
print("Kaikki pitkät vokaalit:", len(pitka_vokaali))
for item in pitka_vokaali:
    print(item)
print("Kaikki muut itäiset eli diftongi:", len(muu_itainen))
for item in muu_itainen:
    print(item)
print("Kaikki muut esiintymät:", len(muu))
for item in muu:
    print(item)